In [ ]:
# make sure we are working in module directory
repo_root = !git rev-parse --show-toplevel
module_path = repo_root[0] + "/backend/heatflask"
%cd $module_path

In [ ]:
# %load Users.py
from logging import getLogger
from DataAPIs import redis, init_collection

"""
***  For Jupyter notebook ***

Paste one of these Jupyter magic directives to the top of a cell
 and run it, to do these things:

  * %%cython --annotate
      Compile and run the cell

  * %load Users.py
     Load Users.py file into this (empty) cell

  * %%writefile Users.py
      Write the contents of this cell to Users.py

"""

log = getLogger(__name__)
log.propagate = True

APP_NAME = "heatflask"
COLLECTION_NAME = "users"
CACHE_PREFIX = "U:"

USER_TTL = 365 * 24 * 3600  # Drop a user after a year of inactivity

DATA = {}


async def get_collection():
    if "col" not in DATA:
        DATA["col"] = await init_collection(
            COLLECTION_NAME, force=False, ttl=USER_TTL, cache_prefix=CACHE_PREFIX
        )
    return DATA["col"]


